<a href="https://colab.research.google.com/github/Sukhwinder9813/EmojiPredictionfromTweet/blob/master/EmojiPredictorfromTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
import emoji

In [0]:
from keras.preprocessing.text import Tokenizer

In [0]:
from nltk.corpus import stopwords


In [0]:
data=pd.read_csv('Train.csv')

In [0]:
x_train=data['TEXT'].values

In [0]:
y_train=data['Label'].values

In [0]:
smoothtweets=[]
stopper=set(stopwords.words('english'))
for tweets in x_train:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothtweets.append(str)

In [0]:
smoothtweets[2]

'Been friends since 7th grade. Look us following dreams love and…\n '

In [0]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', split=" ", lower=True)
tokenizer.fit_on_texts(smoothtweets)
newsmoothtweets=[]
newsmoothtweets=tokenizer.texts_to_sequences(smoothtweets)

In [0]:
print(newsmoothtweets[3])

[15, 378, 23, 436, 325, 10222, 161, 1006, 470, 10223]


In [0]:
from keras.preprocessing import sequence
X_train=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [0]:
from keras.layers import *
from keras.models import Sequential 

In [0]:
from keras.utils import to_categorical

In [0]:
Y_train=to_categorical(y_train)

In [0]:
Y_train.shape

(70000, 20)

In [0]:
X_train.shape

(70000, 12)

In [0]:
wordembedd={}
f=open('/content/glove.6B.50d.txt')
for line in f:
  words=line.split()
  word=words[0]
  coefs=np.asarray(words[1:],dtype='float')
  wordembedd[word]=coefs
f.close()

In [0]:
def populate_weight_matrix(vocab, raw_embedding):
    # Create weight matrix from pre-trained embeddings
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, 50))
    for word, i in vocab.items():
        if word in raw_embedding:
            weight_matrix[i] = raw_embedding[word]
    return weight_matrix

In [0]:
vocab=tokenizer.word_index

In [0]:
len(vocab)
import math

In [0]:
weight_matrix=populate_weight_matrix(vocab,wordembedd)

In [0]:
weight_matrix[8]

array([ 0.092086,  0.2571  , -0.58693 , -0.37029 ,  1.0828  , -0.55466 ,
       -0.78142 ,  0.58696 , -0.58714 ,  0.46318 , -0.11267 ,  0.2606  ,
       -0.26928 , -0.072466,  1.247   ,  0.30571 ,  0.56731 ,  0.30509 ,
       -0.050312, -0.64443 , -0.54513 ,  0.86429 ,  0.20914 ,  0.56334 ,
        1.1228  , -1.0516  , -0.78105 ,  0.29656 ,  0.7261  , -0.61392 ,
        2.4225  ,  1.0142  , -0.17753 ,  0.4147  , -0.12966 , -0.47064 ,
        0.3807  ,  0.16309 , -0.323   , -0.77899 , -0.42473 , -0.30826 ,
       -0.42242 ,  0.055069,  0.38267 ,  0.037415, -0.4302  , -0.39442 ,
        0.10511 ,  0.87286 ])

In [0]:
max_length = math.ceil(sum([len(s.split(" ")) for s in smoothtweets])/len(smoothtweets))
vocab_size=len(vocab)+1
model=Sequential()
model.add(Embedding(vocab_size,50,weights=[weight_matrix],input_length=max_length+2,trainable=True,))
model.add(LSTM(128,dropout=0.2,return_sequences=True))
model.add(LSTM(128,dropout=0.2))
model.add(Dense(20,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 12, 50)            4275000   
_________________________________________________________________
lstm_34 (LSTM)               (None, 12, 128)           91648     
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 20)                2580      
Total params: 4,500,812
Trainable params: 4,500,812
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(X_train,Y_train,epochs=5,batch_size=128,shuffle=True,validation_split=0.15)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 59500 samples, validate on 10500 samples
Epoch 1/5
59500/59500 [==============================] - 25s 423us/step - loss: 2.5389 - accuracy: 0.2625 - val_loss: 2.3151 - val_accuracy: 0.3107
Epoch 2/5
59500/59500 [==============================] - 24s 401us/step - loss: 2.2685 - accuracy: 0.3238 - val_loss: 2.2342 - val_accuracy: 0.3350
Epoch 3/5
59500/59500 [==============================] - 23s 384us/step - loss: 2.1415 - accuracy: 0.3601 - val_loss: 2.2115 - val_accuracy: 0.3427
Epoch 4/5
59500/59500 [==============================] - 23s 385us/step - loss: 1.9961 - accuracy: 0.4021 - val_loss: 2.2347 - val_accuracy: 0.3344
Epoch 5/5
59500/59500 [==============================] - 23s 385us/step - loss: 1.8113 - accuracy: 0.4527 - val_loss: 2.3166 - val_accuracy: 0.3407


In [0]:
model.evaluate(X_train,Y_train)

70000/70000 [==============================] - 11s 158us/step


[1.6350010813031879, 0.5063571333885193]

In [0]:
model.predict_classes(X_train[1:13])

array([1, 9, 3, 3, 7, 5, 6, 2, 7, 9, 9, 1])

In [0]:
smoothtweets[9]

'Story On Saturday 136 Collins Ave, Miami Beach, FL 33139 Ladies Contact Me To Join Me In my…\n '

In [0]:
df=pd.read_csv('Mapping.csv')

In [0]:
df.head(20)

,Unnamed: 0,emoticons,number
0,0,😜,0
1,1,📸,1
2,2,😍,2
3,3,😂,3
4,4,😉,4
5,5,🎄,5
6,6,📷,6
7,7,🔥,7
8,8,😘,8
9,9,❤,9


In [0]:
data['Label'][9]

7

In [0]:
df3=pd.read_csv('Test.csv')

In [0]:
df3.head()

,Unnamed: 0,id,TEXT
0,0,0,Thought this was cool...#Repost (get_repost)・・...
1,1,1,Happy 4th! Corte madera parade. #everytownusa ...
2,2,2,Luv. Or at least something close to it. @ Unio...
3,3,3,There's a slice of pie under that whipped crea...
4,5,5,#thankyou for your thank you We adore you both...


In [0]:
xtest=df3['TEXT']

In [0]:
smoothTESTtweets=[]
stopper=set(stopwords.words('english'))
for tweets in xtest:
  words=tweets.split(" ")
  str = ""
  for word in words:
      if word[0] != "@" and word not in stopper:
          if word[0] == "#":
              word = word[1:]
          str += word + " "
  smoothTESTtweets.append(str)

In [0]:
newsmoothtweets=tokenizer.texts_to_sequences(smoothTESTtweets)

In [0]:
newsmoothtweets[2]

[1201, 887, 1092, 411, 797, 45, 727, 474, 1587, 298]

In [0]:
x_test=sequence.pad_sequences(newsmoothtweets,maxlen=12,padding='post')

In [0]:
x_test[3]

array([ 667, 7371, 1648, 5277,  722,   86, 6889,    0,    0,    0,    0,
          0], dtype=int32)

In [0]:
data=model.predict_classes(x_test)


In [0]:
for i in range(20,35,1):
  print(df['emoticons'][data[i]])
  print(smoothTESTtweets[i])

😂
One favorite gyms FREE Fucking membership, get ass . Balboa Park
 
😍
Hanging guy today Thank great shoot filled plenty of…
 
😎
mood today Parkway North High School
 
😍
Chandelier Widener Library
 
😎
Sorry jump bandwagon. thisfilteriseverything Las…
 
❤
️ Princeton, New Jersey
 
❤
My main man Gladstone, Missouri
 
😂
Eat shit Irma ghostriotradio hurricaneirma irmanators goodforsprinkle pbr…
 
❤
Max went first aquarium trip went moving walkway. He amazing! ️ Check out…
 
😍
ikea fun me ... oh dam store, like disney interior architect's IKEA
 
😂
It's like I blinked 2 years flew Short Pump Town Center
 
❤
When moment meet future husband!!! marlonovershawn teammarlon Stress…
 
😎
We working hard today McDonald's 16827 Marsh Rd
 
🔥
"No Flocking" shot coming soon! LSV2 available datpiff freestyle…
 
🔥
Dj Ivan G! One Mixer, One Man, One Dj, That Rocks The Party! Always Mixes for…
 


In [0]:
smoothTESTtweets[2]

'Luv. Or least something close it. Union Hill, Richmond, Virginia\n '

['Luv. Or least something close it. Union Hill, Richmond, Virginia\n ',
 "There's slice pie whipped cream. HouseofPies House Pies\n ",
 'thankyou thank We adore + plan moreeeee! Hosting wedding was…\n ',
 'SPECIAL4U Lyric video posted youtube channel today 6PM EST ! Z…\n ',
 "Momma Tanya's In town ! Awesome dinner friends! Perch\n ",
 'Thing 1 Thing 2 Huron, Ohio\n ',
 'Bday girl random Sheraton New York Times Square\n ',
 'Always fun forever wedding date Congrats &amp; ',
 'La La Land Griffith Park Observatory-Los Angeles ,CA\n ',
 'Friends. Seattle Seattle, Washington\n ',
 'GETIT GOTIT GOOD WHATEVERIWANT BOW Oakland, California\n ',
 'Hanging bestie day ️#mtlove hyalite daysoff fishing endofsummer…\n ',
 'Stoked sitting tonight!! Therese folks Great…\n ']